In [13]:
import sys
from herbie import Herbie
from datetime import datetime, timedelta
import pygrib
import numpy as np
import pandas as pd
import os

In [14]:
parameters = np.linspace(1, 150, num=150, dtype=int)
date = "20230201"
dt = date + " 10:00"
save_dir = '/Users/kkjesus/data/'
pred_hours = 1


In [15]:
H2 = Herbie(
    dt,
    model="hrrr",
    product="nat",
    save_dir=save_dir,
    fxx=pred_hours,
    verbose=True,
    overwrite=False
).download(":APCP:surface:")
    # .download()

In [12]:
grib_path = '/Users/kkjesus/data/hrrr/20230201/subset_85e4fa12__hrrr.t10z.wrfnatf01.grib2'
grib = pygrib.open(grib_path)
print(grib)
print("grib")
# H2.tell_me_everything()
for g in grib:
    print(g)
    # lats, lons = g.latlons()
    # values = g.values
    # print(lats.min(), lats.max(), lons.min(), lons.max())
    # print(values.shape, values.min(), values.max())

In [9]:
for p in parameters:
    try:
        grib_msgs = grib[int(p)]
    except OSError:
        print("Not that many messages")
        break
    print("Layer: %s Name: %s  Units: %s  Level: %s (%s)" % (
    p, grib_msgs.name, grib_msgs.units, grib_msgs.level, grib_msgs.typeOfLevel))
    lat_st_mid = 30.2241
    lon_st_mid = -92.0198
    data = grib_msgs.values
    lt, ln = grib_msgs.latlons()
    st_lt_m = np.full_like(lt, lat_st_mid)
    st_ln_m = np.full_like(ln, lon_st_mid)
    dis_mat = (lt - st_lt_m) ** 2 + (ln - st_ln_m) ** 2
    p_lt, p_ln = np.unravel_index(dis_mat.argmin(), dis_mat.shape)
    value = data[p_lt, p_ln]
    print(lt[p_lt][p_ln])
    print(ln[p_lt,p_ln])
    print(p_lt, " ", p_ln)
    print()
    print(value)
    print(data, data.shape)

Layer: 1 Name: Total Precipitation  Units: kg m**-2  Level: 0 (surface)
30.23591780027958
-92.03036502874882
227   1076

0.001
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] (1059, 1799)
Not that many messages


In [6]:
# df = {
#     "car" : [1, 2, 3],
#     "boat": [4, 5, 6],
# }
df = pd.DataFrame({
    "car" : [1, 2, 3],
    "boat": [4, 5, 6],
    "george": [10,11,12]
})

In [7]:
# df = pd.concat([df, pd.Series([7,8,9])], ignore_index=True, axis=0, join='outer')
df2 = pd.DataFrame({
    "car" : [11, 22, 33],
    "boat" : [44,55,66],
    "george" : [77,88,99]
})
df = df.merge(df2, how='outer')
print(df)

   car  boat  george
0    1     4      10
1    2     5      11
2    3     6      12
3   11    44      77
4   22    55      88
5   33    66      99


In [23]:
print(df['car'][0], df['car'][len(df)-1])

1 33


In [8]:
dir = "/home/kaleb/Desktop/WRFextract_2-3/Hourly/"
filepathsep = dir.split('/')
hourly_file_path = ''
for i in range(len(filepathsep) - 1):
    if filepathsep[i].rfind("Hourly") != -1:
        break
    hourly_file_path += filepathsep[i] + '/'
hourly_file_path += "Daily_Monthly/"
print(hourly_file_path)

/home/kaleb/Desktop/WRFextract_2-3/Daily_Monthly/


In [9]:
df_states = {}
for fips_folder in sorted(os.listdir(hourly_file_path)):
    fips_dir = hourly_file_path + fips_folder + '/'
    # print(fips_dir)
    for monthly_file in sorted(os.listdir(fips_dir)):
        # print(monthly_file)
        full_path = fips_dir + monthly_file
        state_fips = monthly_file[5:7]
        state_abbrev = monthly_file[11:13]
        year = monthly_file[14:18]
        month = monthly_file[18:20]
        col_name = state_fips + '_' + year + '_' + month
        if not col_name in df_states:
            df_states[col_name] = []
        df_states[col_name].append(full_path)
        # try:
        #     df_states[col_name].loc[0] = full_path
        # except:
        #     df_states.index = df_states.index + 1
        #     df_states[col_name].loc[0] = full_path
        # prints empty df
        # df_states = df_states.append({col_name:full_path}, ignore_index=True)
        # df_states = df.merge()
print(df_states)


FileNotFoundError: [Errno 2] No such file or directory: '/home/kaleb/Desktop/WRFextract_2-3/Daily_Monthly/'

In [18]:
for col in df_states:
    print(col)
    for dir in df_states[col]:
        print(dir)
    in_file_path_sep = df_states[col][0].split('/')
    needed_info_sep = in_file_path_sep[-1].split('_')
    print(needed_info_sep)
    break

17_2023_01
/home/kaleb/Desktop/WRFextract_2-3/Daily_Monthly/17091/HRRR_17091_IL_202301.csv
/home/kaleb/Desktop/WRFextract_2-3/Daily_Monthly/17155/HRRR_17155_IL_202301.csv
['HRRR', '17091', 'IL', '202301.csv']


In [10]:
np.zeros(len(df))

array([0., 0., 0., 0., 0., 0.])